In [496]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import scipy.linalg as sci
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

In [497]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Parameter 초기화

In [498]:
## Learning parameters
initial_run = 1 #1: starts training from scratch; 0: resumes training 
n_epochs = 5000 #Number of training epochs, for observing the current performance set it to 0
learning_rate = 0.0005 #Learning rate

batch_size = 1024 #Mini-batch size
test_size = 10000 #Size of the validation/test set
batch_per_epoch = 20 #Numbers of mini-batches per epoch

anneal_param = 1.0 #Initial annealing parmeter
annealing_rate = 1.001 #Annealing rate

In [499]:
## System parameters
M = 64 #Number of BS antennas
P = 1 #Power
K =  2 #Number of users
L = 8 #Number of pilots
Lp = 2 #Number of paths
B = 30 #Number of feedback bits per user

## Limited scattering channel parameters
LSF_UE = np.array([0.0,0.0],dtype=np.float32) #Mean of path gains for K users
Mainlobe_UE= np.array([0,0],dtype=np.float32) #Center of the AoD range for K users
HalfBW_UE = np.array([30.0,30.0],dtype=np.float32) #Half of the AoD range for K users

# SNR
snr_dl = 10 #SNR in dB
noise_std_dl = np.float32(np.sqrt(1/2)*np.sqrt(P/10**(snr_dl/10))) #STD of the Gaussian noise (per real dim.)

### Pilot Sequence 초기화 -- DFT Matrix 이용

In [500]:
DFT_Matrix = sci.dft(M) 
X_init = DFT_Matrix[0::int(np.ceil(M/L)),:] 
Xp_init = np.sqrt(P/M)*X_init
Xp_r_init = torch.Tensor(np.float32(np.real(Xp_init))).to(device)
Xp_i_init = torch.Tensor(np.float32(np.imag(Xp_init))).to(device)
## Pilot sequence cuda 지정 완료

### Matrix 연산 함수

In [501]:
def mult_mod(M, N, left_right):
    tensor_shape = M.shape
    dims = N.shape 

    if left_right == 'r':
        # M: (batch_size, n, m) N: (m, p)
        n = tensor_shape[1]
        m = dims[0]
        p = dims[1]

        # PyTorch의 행렬 곱
        y = torch.reshape(torch.matmul(M.view(-1, m), N), (-1, n, p))

    elif left_right == 'l':
        # M: (batch_size, n, m)
        # N: (p, n)
        m = tensor_shape[2]
        p = dims[0]
        n = dims[1]

        # PyTorch에서는 `permute()`로 전치 가능
        MT = torch.Tensor(M).permute(0, 2, 1)  # (batch_size, m, n)
        NT = N.T  # (n, p)

        MTNT = torch.reshape(torch.matmul(MT.view(-1, n), NT), (-1, m, p))
        y = MTNT.permute(0, 2, 1)  # (batch_size, n, p)로 변환

    return y.to(device)

def mult_mod_complex(Mr, Mi, Nr, Ni, left_right):
    yr = mult_mod(Mr, Nr, left_right) - mult_mod(Mi, Ni, left_right)
    yi = mult_mod(Mr, Ni, left_right) + mult_mod(Mi, Nr, left_right)
    return yr.to(device), yi.to(device)

### Batch data 생성 함수

In [502]:
## 리스트 인덱싱은 [층, 행, 열], [행, 열]임!
def generate_batch_data(batch_size,M,K,
                        Lp,#number of paths
                        LSF_UE #Mean of path gains for K users
                        ,Mainlobe_UE #Center of the AoD range for K users
                        ,HalfBW_UE #Half of the AoD range for K users
                        ):
    alphaR_input = np.zeros((batch_size,Lp,K))
    alphaI_input = np.zeros((batch_size,Lp,K))
    theta_input = np.zeros((batch_size,Lp,K))
    for kk in range(K): # for the number of users
        alphaR_input[:,:,kk] = np.random.normal(loc=LSF_UE[kk], scale=1.0/np.sqrt(2), size=[batch_size,Lp])
        alphaI_input[:,:,kk] = np.random.normal(loc=LSF_UE[kk], scale=1.0/np.sqrt(2), size=[batch_size,Lp])
        theta_input[:,:,kk] = np.random.uniform(low=Mainlobe_UE[kk]-HalfBW_UE[kk], high=Mainlobe_UE[kk]+HalfBW_UE[kk], size=[batch_size,Lp])
 
    #### Actual Channel
    from0toM = np.float32(np.arange(0, M, 1))
    alpha_act = alphaR_input + 1j*alphaI_input
    theta_act = (np.pi/180)*theta_input
    
    h_act = np.complex64(np.zeros((batch_size,M,K)))
    hR_act = np.float32(np.zeros((batch_size,M,K)))
    hI_act = np.float32(np.zeros((batch_size,M,K)))
    
    for kk in range(K):
        for ll in range(Lp):
            theta_act_expanded_temp = np.tile(np.reshape(theta_act[:,ll,kk],[-1,1]),(1,M))
            response_temp = np.exp(1j*np.pi*np.multiply(np.sin(theta_act_expanded_temp),from0toM))
            alpha_temp = np.reshape(alpha_act[:,ll,kk],[-1,1])
            h_act[:,:,kk] += (1/np.sqrt(Lp))*alpha_temp*response_temp
        hR_act[:,:,kk] = np.real(h_act[:,:,kk])
        hI_act[:,:,kk] = np.imag(h_act[:,:,kk])
        
    h_act = torch.tensor(h_act).to(device)
    hR_act = torch.tensor(hR_act).to(device)
    hI_act = torch.tensor(hI_act).to(device)
        
    return(h_act, hR_act, hI_act)

In [503]:
# 입력 데이터 텐서
hR = torch.randn((batch_size, M, K), dtype=torch.float32)  # 실수 채널 행렬
hI = torch.randn((batch_size, M, K), dtype=torch.float32)  # 허수 채널 행렬

### Downlink Pilot Training -- Pilot Sequence as DNN parameters

In [504]:
## hR, hI 만 cuda로 들어가면 됨됨
class DLTrainingPhase(nn.Module):
    def __init__(self, P, noise_std_dl):
        super(DLTrainingPhase, self).__init__()
        
        # noise/annealing parameter
        self.noise_std = torch.tensor(noise_std_dl, dtype=torch.float32).to(device)
        self.aneal = torch.tensor(1.0, dtype=torch.float32).to(device)
        
        # Pilot sequence - Use pre-initialized values
        self.Xp_r = nn.Parameter(Xp_r_init.clone().to(device))
        self.Xp_i = nn.Parameter(Xp_i_init.clone().to(device))
        
        # Power normalizing
        self.P = P
        self.normalize_pilot()
        
    def normalize_pilot(self):
        # Function : Normalizing the pilot sequence vectors
        norm_X = torch.sqrt(torch.sum(self.Xp_r**2 + self.Xp_i**2, dim = 1, keepdim = True)) # (. , * , . ) *에 대해 sum 수행
        self.Xp_r.data = torch.sqrt(torch.tensor(self.P)) * (self.Xp_r / norm_X)   
        self.Xp_i.data = torch.sqrt(torch.tensor(self.P)) * (self.Xp_i / norm_X)
        
    def forward(self, hR, hI, K, M, L):
        y_nless = {}
        y_noisy = {}
        
        for kk in range(K):
            hR_temp = hR[:, :, kk].reshape(-1, M, 1) # 차원 (batch_size, M)
            hI_temp = hI[:, :, kk].reshape(-1, M, 1)

            # 복소수 행렬 곱 수행
            y_nless_r, y_nless_i = mult_mod_complex(hR_temp, hI_temp, self.Xp_r, self.Xp_i, 'l')

            # 실수 및 허수 결합 -> 복소수로 결합 아니고 real representation
            y_nless[kk] = torch.cat([y_nless_r.view(-1, L), y_nless_i.view(-1, L)], dim=1)

            # 가우시안 노이즈 추가
            noise = torch.randn_like(y_nless[kk]) * self.noise_std
            y_noisy[kk] = y_nless[kk] + noise
        
        return y_noisy

### UE side DNN - Quantizer for CSI feedback

In [505]:
class UE_DNN(nn.Module):
    def __init__(self, L, B, K, anneal = 1.0):
        super(UE_DNN, self).__init__()
        self.anneal = anneal
        self.input_dim = 2*L
        self.K=K
        
        self.model = nn.Sequential(
            nn.BatchNorm1d(self.input_dim),
            nn.Linear(self.input_dim, 1024),
            nn.ReLU(),
            
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(),
            
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            
            nn.BatchNorm1d(256),
            nn.Linear(256, B)
        )
        self.model.to(device)
        
    def forward(self, x):
        InfoBits = {0:0}
        for kk in range(self.K):
            InfoBits_linear = self.model(x[kk])  # 신경망을 통과한 값

            # Straight-Through Estimator (STE) 적용
            InfoBits_tanh = torch.tanh(self.anneal * InfoBits_linear)
            InfoBits_sign = torch.sign(InfoBits_linear)

            # Forward: Sign 값을 사용, Backward: Tanh gradient 사용
            InfoBits[kk] = InfoBits_tanh + (InfoBits_sign - InfoBits_tanh).detach()
        
        return InfoBits

### BS side DNN - Seperately estimating channel hhat_k

##### [1] Precoder, Rate 계산 함수

In [506]:
## Precoder
## 주의 : 입력을 H^H로 받았다고 취급함! 
def ZF_Precoding(h_hat):
    H_hat = h_hat
    V = torch.zeros((H_hat.shape[0], H_hat.shape[1], H_hat.shape[2]), dtype=torch.complex64)
    
    for i in range(H_hat.shape[0]):
        V[i, :, :] = torch.linalg.pinv(H_hat[i, :, :].T)
        V[i, :, :] = V[i, :, :] / torch.sqrt(torch.trace(V[i, :, :] @ (V[i, :, :].H)))
    
    return V.to(device)
    
def MRT_Precoding(h_hat):
    H_hat = h_hat
    V = torch.zeros((H_hat.shape[0], H_hat.shape[1], H_hat.shape[2]), dtype=torch.complex64)
    
    for i in range(H_hat.shape[0]):
        V[i, :, :] = H_hat[i, :, :].conj()
        V[i, :, :] = V[i, :, :] / torch.sqrt(torch.trace(V[i, :, :] @ (V[i, :, :].H)))
    
    return V.to(device)

In [507]:
def rate_calc(h_act_user, M, K, k_idx, V, noise_std):
    H_act = h_act_user
    nom_plus_denom = torch.zeros((H_act.shape[0], 1)).to(device) + torch.tensor(2 * noise_std ** 2).to(device)
    
    for kk in range(K):
        product = torch.bmm(H_act.clone().unsqueeze(-1).permute(0, 2, 1), V[:, :, kk].clone().unsqueeze(-1)).squeeze(-1)
        norm2 = torch.abs(product) ** 2
        norm2.to(device)
        
        if kk == k_idx:
            nom = norm2 # Wanted signal power
        nom_plus_denom += norm2
        
    denom = nom_plus_denom - nom
    rate = torch.mean(torch.log2(1 + (nom / denom)))
    
    return rate

##### [2] Single BS DNN

In [508]:
class SSC_BS_Independent_NN(nn.Module):
    def __init__(self, B, M):
        super(SSC_BS_Independent_NN, self).__init__()
        self.input_dim = B
        self.M = M

        self.model = nn.Sequential(
            nn.BatchNorm1d(self.input_dim),
            nn.Linear(self.input_dim, 1024),
            nn.ReLU(),
            
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(),
            
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            
            nn.BatchNorm1d(256),
            nn.Linear(256, 2*M)          
        )
        self.model.to(device)
        
    def forward(self, q):
        hhat_k_R = self.model(q)[:, :self.M]
        hhat_k_I = self.model(q)[:, self.M:]
        hhat_k = torch.tensor(hhat_k_R + 1j* hhat_k_I).to(device)
        return hhat_k

In [566]:
class SSC_BS_DNN(nn.Module):
    def __init__(self, B, K, M):
        super(SSC_BS_DNN, self).__init__()
        self.num_users = K
        self.quant_bit = B
        self.tantenna = M
        
        self.dnn_list = nn.ModuleList([
            SSC_BS_Independent_NN(self.quant_bit, self.tantenna) for _ in range(self.num_users)
        ])
        
    def forward(self, Q, h_test, noise_std, MRT = True):
        hhat_k = [self.dnn_list[i](Q[i]) for i in range(self.num_users)]
        Hhat = torch.stack(hhat_k, dim=1) # (batch, M, K) 차원으로 concatenate
        
        rate= {}
        
        if MRT == True:
            for kk in range(self.num_users):
                rate[kk] = self.compute_rate_MRT(Hhat, h_test[:, :, kk], self.num_users, kk, noise_std)
        else:
            for kk in range(self.num_users):
                rate[kk] = self.compute_rate_ZF(Hhat, h_test[:, :, kk], self.num_users, kk, noise_std)
            
        return rate
    
    def compute_rate_ZF(self, Hhat, h_act_kk, K, k_idx, noise_std):
        V = torch.zeros((Hhat.shape[0], Hhat.shape[2], Hhat.shape[1]), dtype = torch.complex64).to(device)
        for i in range(Hhat.shape[0]):
            V[i, :, :] = torch.linalg.pinv(Hhat[i, :, :])
            V[i, :, :] = V[i, :, :] / torch.sqrt(torch.trace(V[i, :, :].H @ V[i, :, :]))
            
        nom_plus_denom = torch.zeros((h_act_kk.shape[0], 1)).to(device) + torch.tensor(2 * noise_std ** 2).to(device)    
        for kk in range(K):
            product = torch.bmm(h_act_kk.clone().unsqueeze(-1).permute(0, 2, 1), V[:, :, kk].clone().unsqueeze(-1)).squeeze(-1)
            norm2 = torch.abs(product) ** 2
            norm2.to(device)
            
            if kk == k_idx:
                nom = norm2
            nom_plus_denom += norm2
            
        denom = nom_plus_denom - nom
        rate = torch.log2(1+ (nom / denom))
        
        return -rate
            
    def compute_rate_MRT(self, Hhat, h_act_kk, K, k_idx, noise_std):
        V = torch.zeros((Hhat.shape[0], Hhat.shape[1], Hhat.shape[2]), dtype = torch.complex64).to(device)
        for i in range(Hhat.shape[0]):
            V[i, :, :] = Hhat[i, :, :]
            V[i, :, :] = V[i, :, :] / torch.sqrt(torch.trace(V[i, :, :].H @ V[i, :, :]))
            
        nom_plus_denom = torch.zeros((h_act_kk.shape[0], 1)).to(device) + torch.tensor(2 * noise_std ** 2).to(device)    
        for kk in range(K):
            product = torch.bmm(h_act_kk.clone().unsqueeze(-1).permute(0, 2, 1).conj(), V[:, :, kk].clone().unsqueeze(-1)).squeeze(-1)
            norm2 = torch.abs(product) ** 2
            norm2.to(device)
            
            if kk == k_idx:
                nom = norm2
            nom_plus_denom += norm2
            
        denom = nom_plus_denom - nom
        rate = torch.log2(1+ (nom / denom))
        
        return -rate

### 모델 학습하기

In [567]:
pilot_train = DLTrainingPhase(P, noise_std_dl)
ue_dnn = UE_DNN(L, B, K, annealing_rate)
bs_dnn = SSC_BS_DNN(B, K, M)

In [568]:
class CustomLoss(nn.Module):
    def __init__(self, K):
        super(CustomLoss, self).__init__()
        self.K = K
        
    def forward(self, rate):
        rate_total = sum(rate[k] for k in range(self.K))
        loss = torch.mean(rate_total)
        
        return loss

Lossfunc = CustomLoss(K)

In [569]:
optimizer = optim.Adam(list(pilot_train.parameters())+list(ue_dnn.parameters())+list(bs_dnn.parameters())
                       , lr = learning_rate)

In [570]:
## Test set 생성
h_test, hR_test, hI_test = generate_batch_data(test_size, M, K, Lp, LSF_UE, Mainlobe_UE, HalfBW_UE)
h_test = h_test.requires_grad_(True)


# Tensor 변환 후 GPU 할당
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hR_test = hR_test.clone().detach().to(device, dtype=torch.float32)
hI_test = hI_test.clone().detach().to(device, dtype=torch.float32)
noise_std_test = torch.tensor(noise_std_dl).clone().detach().to(device, dtype=torch.float32)

In [571]:
AA = sio.loadmat('Data_test_K2M64Lp2L8_withParams.mat') # -> 학습이 끝난 후 loss를 계산하기 위한 data
hR_test_Final = AA['hR_act_test_Final']
hI_test_Final = AA['hI_act_test_Final']
h_test_Final = AA['h_act_test_Final']

hR_test_Final = torch.tensor(hR_test_Final, dtype=torch.float32).to(device)
hI_test_Final = torch.tensor(hI_test_Final, dtype=torch.float32).to(device)
h_test_Final = torch.tensor(h_test_Final, dtype=torch.complex64).to(device)

In [572]:
print(h_test.size())

torch.Size([10000, 64, 2])


In [573]:
# 초기 모델 저장 경로 설정
save_path = './params.pth'

# 모델 불러오기 (처음 실행 여부 확인)
initial_run = 1
if initial_run == 0:
    checkpoint = torch.load(save_path)
    pilot_train.load_state_dict(checkpoint['pilot_train'])
    bs_dnn.load_state_dict(checkpoint['bs_dnn'])
    ue_dnn.load_state_dict(checkpoint['ue_dnn'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    best_loss = checkpoint['best_loss']
    print("✅ 모델 파라미터 불러오기 완료!")
else:
    best_loss = float('inf')  # 초기 Best Loss 설정

In [574]:
# Loop for ZF
for epoch in range(n_epochs):
    optimizer.zero_grad()
    
    for _ in range(batch_per_epoch):
        # train dataset 생성
        h_batch, hR_batch, hI_batch = generate_batch_data(batch_size,M,K,Lp,LSF_UE,Mainlobe_UE,HalfBW_UE)
        hR_batch = hR_batch.requires_grad_(True)
        hI_batch = hI_batch.requires_grad_(True)
        h_batch = h_batch.requires_grad_(True)
        
        y_pilot = pilot_train(hR_batch, hI_batch, K, M, L) # K user의 파일럿 수신 신호 리스트
        UE_Feedback = ue_dnn(y_pilot)
        rate_ZF = bs_dnn(UE_Feedback, h_batch, noise_std_dl, MRT=False)
        loss_ZF= Lossfunc(rate_ZF)
        
        loss_ZF.backward()
        optimizer.step
        
    if epoch % 10 == 0:
        y_pilot = pilot_train(hR_test, hI_test, K, M, L)
        UE_Feedback = ue_dnn(y_pilot)
        rate_ZF = bs_dnn(UE_Feedback, h_test, noise_std_dl, MRT = False)
        loss_ZF = Lossfunc(rate_ZF)
        
        if loss_ZF < best_loss:
            best_loss = loss_ZF
            
            torch.save({
                'pilot_train' : pilot_train.state_dict(),
                'bs_dnn' : bs_dnn.state_dict(),
                'ue_dnn' : ue_dnn.state_dict(),
                'optimizer' : optimizer.state_dict(),
                'best_loss' : best_loss
            }, save_path)
            
            print(f"✅ Model saved at epoch {epoch}, Loss: {best_loss:.5f}")
        else:
            anneal_param = anneal_param*annealing_rate
        print('epoch',epoch,' anneal_param:%4.4f'%anneal_param)
        print('         loss_test:%2.5f'%-loss_ZF,'  best_test:%2.5f'%-best_loss)
        
y_pilot = pilot_train(hR_test_Final, hI_test_Final, K, M, L)
UE_Feedback = ue_dnn(y_pilot)
rate_ZF_Final, _ = bs_dnn(UE_Feedback, h_test_Final, noise_std_dl, MRT = False)
loss_ZF_Final = Lossfunc(rate_ZF_Final)

print(-loss_ZF_Final)    


C:\Users\unist\AppData\Local\Temp\ipykernel_45992\1883434635.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hhat_k = torch.tensor(hhat_k_R + 1j* hhat_k_I).to(device)


✅ Model saved at epoch 0, Loss: -1.86619
epoch 0  anneal_param:1.0161
         loss_test:1.86619   best_test:1.86619
✅ Model saved at epoch 10, Loss: -1.88536
epoch 10  anneal_param:1.0161
         loss_test:1.88536   best_test:1.88536
epoch 20  anneal_param:1.0171
         loss_test:1.87854   best_test:1.88536
epoch 30  anneal_param:1.0182
         loss_test:1.86746   best_test:1.88536
epoch 40  anneal_param:1.0192
         loss_test:1.88228   best_test:1.88536
epoch 50  anneal_param:1.0202
         loss_test:1.87547   best_test:1.88536
epoch 60  anneal_param:1.0212
         loss_test:1.85930   best_test:1.88536
epoch 70  anneal_param:1.0222
         loss_test:1.87419   best_test:1.88536
epoch 80  anneal_param:1.0233
         loss_test:1.87098   best_test:1.88536
epoch 90  anneal_param:1.0243
         loss_test:1.86506   best_test:1.88536
epoch 100  anneal_param:1.0253
         loss_test:1.86934   best_test:1.88536
epoch 110  anneal_param:1.0263
         loss_test:1.88506   best_test:

KeyboardInterrupt: 

In [ ]:
# 초기 모델 저장 경로 설정
save_path = './params.pth'

# 모델 불러오기 (처음 실행 여부 확인)
initial_run = 1
if initial_run == 0:
    checkpoint = torch.load(save_path)
    pilot_train.load_state_dict(checkpoint['pilot_train'])
    bs_dnn.load_state_dict(checkpoint['bs_dnn'])
    ue_dnn.load_state_dict(checkpoint['ue_dnn'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    best_loss = checkpoint['best_loss']
    print("✅ 모델 파라미터 불러오기 완료!")
else:
    best_loss = float('inf')  # 초기 Best Loss 설정

In [ ]:
# Loop for MRT
for epoch in range(n_epochs):
    optimizer.zero_grad()
    
    for _ in range(batch_per_epoch):
        # train dataset 생성
        h_batch, hR_batch, hI_batch = generate_batch_data(batch_size,M,K,Lp,LSF_UE,Mainlobe_UE,HalfBW_UE)
        
        y_pilot = pilot_train(hR_batch, hI_batch, K, M, L) # K user의 파일럿 수신 신호 리스트
        UE_Feedback = ue_dnn(y_pilot)
        rate_MRT = bs_dnn(UE_Feedback, h_batch, noise_std_dl, MRT = True)
        loss_MRT = Lossfunc(rate_MRT)
        
        loss_MRT.backward()
        optimizer.step
        
        if epoch % 10 == 0:
            y_pilot = pilot_train(hR_test, hI_test, K, M, L)
            UE_Feedback = ue_dnn(y_pilot)
            rate_MRT = bs_dnn(UE_Feedback, h_test, noise_std_dl, MRT = True)
            loss_MRT = Lossfunc(rate_MRT)
        
        if loss_ZF < best_loss:
            best_loss = loss_ZF
            
            torch.save({
                'pilot_train' : pilot_train.state_dict(),
                'bs_dnn' : bs_dnn.state_dict(),
                'ue_dnn' : ue_dnn.state_dict(),
                'optimizer' : optimizer.state_dict(),
                'best_loss' : best_loss
            }, save_path)
            
            print(f"✅ Model saved at epoch {epoch}, Loss: {best_loss:.5f}")
        else:
            anneal_param = anneal_param*annealing_rate
        print('epoch',epoch,' anneal_param:%4.4f'%anneal_param)
        print('         loss_test:%2.5f'%-loss_ZF,'  best_test:%2.5f'%-best_loss)
        
y_pilot = pilot_train(hR_test_Final, hI_test_Final, K, M, L)
UE_Feedback = ue_dnn(y_pilot)
rate_MRT = bs_dnn(UE_Feedback, h_test_Final, noise_std_dl, MRT = True)
loss_MRT = Lossfunc(rate_MRT)

print(-loss_ZF_Final)

C:\Users\unist\AppData\Local\Temp\ipykernel_27844\1883434635.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hhat_k = torch.tensor(hhat_k_R + 1j* hhat_k_I).to(device)


TypeError: SSC_BS_DNN.compute_rate_ZF() takes 5 positional arguments but 6 were given

In [ ]:
print(AA.keys())

dict_keys(['__header__', '__version__', '__globals__', 'h_act_test_Final', 'hR_act_test_Final', 'hI_act_test_Final', 'alpha_act_test_Final', 'theta_act_test_Final'])
